In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import SparkSession
import os

import sys

spark_path = "D:\spark-3.5.1-bin-hadoop3"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext("local", "test")

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df_p_c = spark.read.csv('product_category.csv', inferSchema=True).toDF("Id","Id_p", "Id_c")
df_c = spark.read.csv('category.csv', inferSchema=True).toDF("Id","Name")
df_p = spark.read.csv('products.csv', inferSchema=True).toDF("Id","Name","Count")

In [5]:
df_p.show()

+---+------------+-----+
| Id|        Name|Count|
+---+------------+-----+
|  1|      Tomato|    4|
|  2|    Cucumber|    3|
|  3|      Orange|   10|
|  4|      Carrot|    5|
|  5|     Cabbage|    2|
|  8|mobile phone|    4|
|  9|         cup|    7|
+---+------------+-----+



In [6]:
df_c.show()

+---+----------+
| Id|      Name|
+---+----------+
|  1|Vegetables|
|  2|    Fruits|
+---+----------+



In [7]:
df_p_c.show()

+---+----+----+
| Id|Id_p|Id_c|
+---+----+----+
|  1|   1|   1|
|  2|   2|   1|
|  3|   4|   1|
|  4|   5|   1|
|  5|   3|   2|
+---+----+----+



In [8]:
df_p.createOrReplaceTempView("products")
df_c.createOrReplaceTempView("category")
df_p_c.createOrReplaceTempView("product_category")

In [9]:
def table(df_product, df_category, df_product_category):
    df_rez = df_product.join(df_product_category, df_product.Id==df_product_category.Id_p, how="left")\
            .join(df_category, df_product_category.Id_c==df_category.Id, how="left")\
            .select(df_product.Name.alias("Product"), df_category.Name.alias("Category"))
    return df_rez

In [10]:
table(df_p, df_c, df_p_c).show()

+------------+----------+
|     Product|  Category|
+------------+----------+
|      Tomato|Vegetables|
|    Cucumber|Vegetables|
|      Orange|    Fruits|
|      Carrot|Vegetables|
|     Cabbage|Vegetables|
|mobile phone|      NULL|
|         cup|      NULL|
+------------+----------+

